In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np

In [2]:
datafile_train=r'\Users\91878\Downloads\Consumer_Complaints_train.csv'
datafile_test=r'\Users\91878\Downloads\Consumer_Complaints_test_share.csv'

cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

In [3]:
cd_test['Consumer disputed?']=np.nan
cd_train['data']='train'
cd_test['data']='test'
cd_all=pd.concat([cd_train,cd_test],axis=0)

In [5]:
cd_all.tail()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data
119601,2012-09-15,Credit card,NaN,Closing/Cancelling account,NaN,NaN,NaN,JPMorgan Chase & Co.,IL,60538,NaN,NaN,Web,2012-09-21,Closed with monetary relief,Yes,NaN,153482,test
119602,2016-07-21,Credit reporting,NaN,Credit reporting company's investigation,No notice of investigation status/result,On XXXX/XXXX/16 a dispute and validation of a ...,Company has responded to the consumer and the ...,Experian,CA,956XX,NaN,Consent provided,Web,2016-07-21,Closed with explanation,Yes,NaN,2023523,test
119603,2015-06-09,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,"FAIR COLLECTIONS & OUTSOURCING, INC.",CA,90803,Servicemember,NaN,Phone,2015-06-12,Closed with explanation,Yes,NaN,1413678,test
119604,2014-02-26,Debt collection,Credit card,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,Encore Capital Group,PA,17764,NaN,NaN,Phone,2014-02-27,Closed with explanation,Yes,NaN,732458,test
119605,2015-02-11,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,NaN,NaN,AllianceOne Recievables Management,CA,92103,NaN,NaN,Web,2015-02-11,Closed with explanation,Yes,NaN,1235226,test


In [6]:
cd_all.shape

(598027, 19)

In [7]:
for col in ['Date received','Date sent to company']:
    cd_all[col]=pd.to_datetime(cd_all[col],infer_datetime_format=True)

In [8]:
cd_all['day_diff']=pd.to_numeric(cd_all['Date sent to company']-cd_all['Date received'])
cd_all.dtypes

Date received                   datetime64[ns]
Product                                 object
Sub-product                             object
Issue                                   object
Sub-issue                               object
Consumer complaint narrative            object
Company public response                 object
Company                                 object
State                                   object
ZIP code                                object
Tags                                    object
Consumer consent provided?              object
Submitted via                           object
Date sent to company            datetime64[ns]
Company response to consumer            object
Timely response?                        object
Consumer disputed?                      object
Complaint ID                             int64
data                                    object
day_diff                                 int64
dtype: object

In [9]:
for col in ['Date received','Date sent to company']:
    cd_all.drop([col],1,inplace=True)

In [10]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 92118
Company public response : 10
Company : 3437
State : 62
ZIP code : 29005
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [11]:
cd_all.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
day_diff                             0
dtype: int64

In [12]:
for col in ['Sub-product','Sub-issue',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_all[varname]=np.where(pd.isnull(cd_all[col]),1,0)
    cd_all.drop([col],1,inplace=True)

In [13]:
cd_all.shape

(598027, 18)

In [14]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())

Product : 12
Issue : 95
Consumer complaint narrative : 92118
Company : 3437
State : 62
ZIP code : 29005
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [15]:
for col in ['ZIP code' ]:
    cd_all.drop([col],1,inplace=True)

cd_all.shape

(598027, 17)

In [16]:
cd_all['Company'].value_counts()

Bank of America                              57887
Wells Fargo & Company                        44337
JPMorgan Chase & Co.                         36040
Equifax                                      36013
Experian                                     33622
                                             ...  
RateBeat LLC                                     1
DCB Financial LLC                                1
Contemporary Information Corp                    1
National Management Recovery Corp. (NMRC)        1
Auto Properties II, LLC                          1
Name: Company, Length: 3437, dtype: int64

In [17]:
for col in ['Company']:
    #print(col,"first loop")
    freqs=cd_all[col].value_counts()
    #print(col,freqs,"first loop frequency")
    k=freqs.index[freqs>1000][:-1]
    print("value of ", k)
    print(k, col,freqs,"first loop frequency k")
    for cat in k:
        name=col+'_'+cat
        cd_all[name]=(cd_all[col]==cat).astype(int)
        #print(name)
    del cd_all[col]
    print(col)

cd_all.shape

value of  Index(['Bank of America', 'Wells Fargo & Company', 'JPMorgan Chase & Co.',
       'Equifax', 'Experian', 'TransUnion Intermediate Holdings, Inc.',
       'Citibank', 'Ocwen', 'Capital One', 'Nationstar Mortgage',
       'Synchrony Financial', 'U.S. Bancorp', 'Ditech Financial LLC',
       'Navient Solutions, Inc.', 'PNC Bank N.A.', 'Encore Capital Group',
       'Amex', 'HSBC North America Holdings Inc.', 'SunTrust Banks, Inc.',
       'Discover', 'TD Bank US Holding Company',
       'Select Portfolio Servicing, Inc',
       'Portfolio Recovery Associates, Inc.', 'Citizens Financial Group, Inc.',
       'Fifth Third Financial Corporation', 'Seterus, Inc.', 'Barclays PLC',
       'ERC', 'BB&T Financial', 'M&T Bank Corporation',
       'PayPal Holdings, Inc.', 'Ally Financial Inc.', 'USAA Savings',
       'Specialized Loan Servicing LLC', 'Regions Financial Corporation',
       'AES/PHEAA', 'Santander Consumer USA Holdings Inc', 'Santander Bank US',
       'Resurgent Capital Se

(598027, 76)

In [18]:
cd_all['Consumer disputed?']=np.where(cd_all['Consumer disputed?']=="Yes",1,0)

In [19]:
k=cd_all['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_all[varname]=np.where(cd_all['Issue']==val,1,0)
    
del cd_all['Issue']
cd_all.shape

(598027, 85)

In [20]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())
    
cd_all.shape

Product : 12
Consumer complaint narrative : 92118
State : 62
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
data : 2


(598027, 85)

In [21]:
k=cd_all['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_all[varname]=np.where(cd_all['State']==val,1,0)
    
del cd_all['State']
cd_all.shape

(598027, 99)

In [22]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_all[col],prefix=col,drop_first=True)
    cd_all=pd.concat([temp,cd_all],1)
    cd_all.drop([col],1,inplace=True)

In [23]:
cd_all.shape

(598027, 118)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [25]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()
my_stop=set(stopwords.words('english')+list(punctuation))

In [26]:
tfidf= TfidfVectorizer(analyzer='word',min_df=0.01,max_df=0.6,
                       stop_words=my_stop, max_features=200)

In [27]:
cd_all['Consumer complaint narrative'].fillna('missing',inplace=True)

In [28]:
tfidf.fit(cd_all['Consumer complaint narrative'])

TfidfVectorizer(max_df=0.6, max_features=200, min_df=0.01,
                stop_words={'!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
                            '+', ',', '-', '.', '/', ':', ';', '<', '=', '>',
                            '?', '@', '[', '\\', ']', '^', '_', '`', 'a',
                            'about', ...})

In [29]:
tfidf_data=pd.DataFrame(data=tfidf.transform(cd_all['Consumer complaint narrative']).toarray(),

                       columns=tfidf.get_feature_names())

In [30]:
cd_all.reset_index(drop=True, inplace=True)
tfidf_data.reset_index(drop=True, inplace=True)

In [31]:
cd_combined = pd.concat([cd_all, tfidf_data], axis=1)

In [32]:
cd_combined.shape

(598027, 304)

In [33]:
del cd_combined['Consumer complaint narrative']

In [34]:
cd_combined.shape

(598027, 303)

In [35]:
cd_train=cd_combined[cd_combined['data']=='train']
del cd_train['data']
cd_test=cd_combined[cd_combined['data']=='test']
cd_test.drop(['Consumer disputed?','data'],axis=1,inplace=True)

In [36]:
del cd_combined

In [37]:
cd_train.shape

(478421, 302)

In [38]:
cd_test.shape

(119606, 301)

In [39]:
x=cd_train.drop(['Consumer disputed?','Complaint ID'],1)
y=cd_train['Consumer disputed?']

In [42]:
#testing the performance of model on 80:20 trai test split

In [43]:
cd_train1,cd_train2=train_test_split(cd_train,test_size=0.2,random_state=2)

In [44]:
x_train1=cd_train1.drop(['Consumer disputed?','Complaint ID'],1)
y_train1=cd_train1['Consumer disputed?']

In [45]:
x_train2=cd_train2.drop(['Consumer disputed?','Complaint ID'],1)
y_train2=cd_train2['Consumer disputed?']

In [ ]:
=========================================================================================

In [ ]:
###grid search

In [65]:
from sklearn.model_selection import GridSearchCV

In [66]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [67]:
grid_search=GridSearchCV(clf,param_grid=params,cv=5,scoring="roc_auc")

In [68]:
grid_search.fit(x,y)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.0000e-02, 1.1112e+02, 2.2223e+02, 3.3334e+02, 4.4445e+02,
       5.5556e+02, 6.6667e+02, 7.7778e+02, 8.8889e+02, 1.0000e+03]),
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [69]:
grid_search.best_estimator_

LogisticRegression(C=666.67, class_weight='balanced')

In [70]:
logr_gs=grid_search.best_estimator_

In [71]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.10f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [72]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.5291933240 (std: 0.00179)
Parameters: {'C': 666.67, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 2
Mean validation score: 0.5068716981 (std: 0.00170)
Parameters: {'C': 1000.0, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: nan (std: nan)
Parameters: {'C': 555.56, 'class_weight': None, 'penalty': 'l1'}

Model with rank: 4
Mean validation score: nan (std: nan)
Parameters: {'C': 555.56, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 5
Mean validation score: nan (std: nan)
Parameters: {'C': 666.67, 'class_weight': 'balanced', 'penalty': 'l1'}



In [73]:
test_score_gs=logr_gs.predict_proba(x_train2)[:,1]

In [74]:
roc_auc_score(y_train2,test_score_gs)

0.5129426544803306

In [104]:
#=============================================================================================

In [ ]:
#Decision tree

In [84]:
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline

In [85]:
from sklearn.tree import DecisionTreeClassifier

In [86]:
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
       }

In [87]:
clf_dt=DecisionTreeClassifier()

In [88]:
random_search=RandomizedSearchCV(clf_dt,cv=10,
                                 param_distributions=params,
                                 scoring='roc_auc',
                                 n_iter=10
                                    )

In [89]:
random_search.fit(x,y)

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(),
                   param_distributions={'class_weight': [None, 'balanced'],
                                        'criterion': ['entropy', 'gini'],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      50, 70],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              20]},
                   scoring='roc_auc')

In [90]:
random_search.cv_results_

{'mean_fit_time': array([25.67983718, 28.40964746, 33.37767551, 22.61489477, 20.93909729,
        33.00738282, 29.91615078, 33.12521605,  6.98241994, 47.79055493]),
 'std_fit_time': array([ 1.57996331,  1.60036522,  2.87147833,  1.62246556,  1.38077211,
         5.19702581, 10.67933309,  2.44860796,  0.41352032, 35.13976249]),
 'mean_score_time': array([0.21212912, 0.15632229, 0.22728083, 0.17581003, 0.15520532,
        0.20028241, 0.14902744, 0.14323297, 0.10008478, 0.15873592]),
 'std_score_time': array([0.06429457, 0.02604556, 0.09080529, 0.03168882, 0.02954759,
        0.13109454, 0.02775518, 0.0166345 , 0.01130036, 0.03053372]),
 'param_min_samples_split': masked_array(data=[15, 15, 2, 2, 5, 20, 20, 10, 2, 5],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[1, 15, 1, 20, 10, 15, 5, 10, 10, 1],
              mask=[False, Fa

In [91]:
random_search.best_estimator_

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=15, min_samples_leaf=10, min_samples_split=5)

In [92]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.6150588989 (std: 0.00339)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 10, 'max_depth': 15, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 2
Mean validation score: 0.6082324351 (std: 0.00315)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 20, 'max_depth': 20, 'criterion': 'gini', 'class_weight': None}

Model with rank: 3
Mean validation score: 0.6056996588 (std: 0.00295)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 1, 'max_depth': 20, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 4
Mean validation score: 0.6056693601 (std: 0.00283)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 5, 'criterion': 'gini', 'class_weight': 'balanced'}

Model with rank: 5
Mean validation score: 0.5943449932 (std: 0.00305)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 15, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': None}



In [93]:
dtree=random_search.best_estimator_

In [94]:
dtree.fit(x,y)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=15, min_samples_leaf=10, min_samples_split=5)

In [95]:
test_score_dt=dtree.predict_proba(x_train2)[:,1]

In [96]:
roc_auc_score(y_train2,test_score_dt)

0.6559146019410693

In [97]:
#clf_dt.fit(x,y)

In [98]:
prediction_dtree=np.where(dtree.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [99]:
submission_dt=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction_dtree))),
                       columns=['Complaint ID','Consumer disputed?'])

In [100]:
import os
os.getcwd()

'C:\\Users\\91878\\Downloads'

In [101]:
submission_dt

,Complaint ID,Consumer disputed?
0,675956,Yes
1,1858795,Yes
2,32637,Yes
3,1731374,No
4,501487,Yes
...,...,...
119601,153482,No
119602,2023523,Yes
119603,1413678,No
119604,732458,No


In [134]:
submission_dt.to_csv('Sanskruti_Zodape_Project 1_part 2.csv',index=False)

In [102]:
test_score_dt=dtree.predict_proba(x)[:,1]

In [103]:
roc_auc_score(y,test_score_dt)

0.6536919529225058

In [ ]:
==============================================================================